In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gamble-dataset/data_v1_preprocessed.csv
/kaggle/input/gamble-dataset/data.csv


In [9]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00:00:01


In [17]:
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [19]:
df = pd.read_csv('/kaggle/input/gamble-dataset/data.csv', sep=';')

In [20]:
df = df[['Comment', 'Label']]

In [21]:
df

,Comment,Label
0,jadi kangen ngebolang di sorong pas PMM kemaren,0
1,Streamer favorit ❤ dan selalu menunggu update ...,1
2,"sketer terus muncul, tiap maen jepek",1
3,Kebelah 2 :),0
4,Jan di spil,0
...,...,...
9292,Bang .. Ikut mendaki bung fiersa atap Negeri t...,0
9293,"Semangat Bosqu , Terbaik ⭐𝗪𝗘𝗧𝗢𝗡𝟴𝟴",1
9294,Penasaran kalo Jefri bener bener ngegambar ses...,0
9295,"@arsep1311 tendang bro gw dukung, sekalian sam...",0


In [28]:
nltk.download('stopwords')

# Initialize stemmer and stopword list
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))  # stopwords bahasa Indonesia

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.strip()  # remove leading/trailing whitespace
    tokens = text.split()  # tokenize
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]  # remove stopwords & lemmatize
    return ' '.join(tokens)

In [30]:
df['Comment'] = df['Comment'].astype(str).apply(preprocess_text)

# modelling

In [11]:
df = pd.read_csv('/kaggle/input/gamble-dataset/data_v1_preprocessed.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9297 entries, 0 to 9296
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  8996 non-null   object
 1   Label    9297 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 145.4+ KB


In [13]:
df = df.dropna()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8996 entries, 0 to 9296
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  8996 non-null   object
 1   Label    8996 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 210.8+ KB


In [15]:
X = df['Comment']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [16]:
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [22]:
# Models to evaluate
models = {
    "Naive Bayes": MultinomialNB(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "SGD": SGDClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
}

In [23]:
# Evaluate each model
for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))


=== Naive Bayes ===
Accuracy: 0.9827777777777778
Confusion Matrix:
 [[1413    5]
 [  26  356]]
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1418
           1       0.99      0.93      0.96       382

    accuracy                           0.98      1800
   macro avg       0.98      0.96      0.97      1800
weighted avg       0.98      0.98      0.98      1800


=== KNN ===
Accuracy: 0.9438888888888889
Confusion Matrix:
 [[1413    5]
 [  96  286]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      1418
           1       0.98      0.75      0.85       382

    accuracy                           0.94      1800
   macro avg       0.96      0.87      0.91      1800
weighted avg       0.95      0.94      0.94      1800


=== Decision Tree ===
Accuracy: 0.9694444444444444
Confusion Matrix:
 [[1381   37]
 [  18  364]]
Classification R